In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
from typing import Tuple
from price_impact.modelling.backtest_analysis import BacktestResults, BacktestSettings
from price_impact.modelling.Futures import FutureContract, Spot
from price_impact.execution.discrete_execution.impact_models import PropagatorImpactModel
from price_impact.execution.discrete_execution.utils import resample_lambdas, example_delta_lambdas, get_std, get_vwap
from price_impact import data_dir, regression_path

from dateutil.relativedelta import relativedelta
import datetime as dt
from pathlib import Path
np.random.seed(0)

PLOT_FOLDER = Path("./plots")
bin_path = data_dir / 'synthetic'/'bins'

plt.style.use('bmh')

In [ ]:
train_months = 2
test_months = 1
smoothing = 'sma20'
half_life_s = 60*60

backtest_settings = BacktestSettings(
        train_len=train_months,
        test_len=test_months,
        bin_freq='10s',
        forecast_horizon_s=10,
        smoothing_str=smoothing,
        decay=half_life_s)

spot = Spot('NQ')
future = FutureContract.from_spot(spot)
freq_m = 15

delta, lambdas = example_delta_lambdas(future, backtest_settings)
print(f"{delta=}")

num_bins = len(lambdas)
trading_hours = num_bins//(60/freq_m)
print(f"{trading_hours=}")
do_bounds = True
spread_bps = 0
stdev = 0.0002
tol = 2e-5
options = {'maxiter': 1e4}

method = 'SLSQP'
total_volume = 0.1
print(f"{total_volume=}")


const_pim = PropagatorImpactModel(half_life_s=half_life_s, delta=delta,
                                  trading_hours=trading_hours, freq_m=freq_m, stdev=stdev,
                                  lambdas=lambdas.mean())

twap = np.ones(shape=num_bins)*total_volume/num_bins
twap_cost_placeholder = const_pim.func_to_optimise(twap)
divisor = twap_cost_placeholder*100

const_res = const_pim.get_optimal_volume(spread_bps=spread_bps, total_volume=total_volume,
                                         do_bounds=do_bounds, tol=tol, options=options,
                                         method=method, divisor=divisor)

var_pim = PropagatorImpactModel(half_life_s=half_life_s, delta=delta,
                                trading_hours=trading_hours, freq_m=freq_m, stdev=stdev,
                                lambdas=lambdas)

var_res = var_pim.get_optimal_volume(spread_bps=spread_bps, total_volume=total_volume,
                                     do_bounds=do_bounds, tol=tol, options=options,
                                     method=method, divisor=divisor)

fig, axs = plt.subplots(2, 1)
idx = np.linspace(0, len(const_res.x)-1, len(const_res.x))

axs[0].bar(idx, const_res.x)
ax1 = axs[0].twinx()
ax1.plot(idx, np.full_like(idx, fill_value=lambdas.mean()), label=r"$\bar{\lambda}$", color='r')
ax1.set_ylim(bottom=0)
ax1.legend(loc=3)

axs[1].bar(idx, var_res.x)
ax2 = axs[1].twinx()
ax2.plot(idx, lambdas, label=r"$\lambda$", color='r')
ax2.set_ylim(bottom=0)
ax2.legend(loc=3)


axs[0].set_ylabel("Volume")
axs[1].set_ylabel("Volume")
ax1.set_ylabel(r"$\lambda$")
ax2.set_ylabel(r"$\lambda$")

ymax = max([axs[i].get_ylim()[1] for i in range(2)])
[axs[i].set_ylim(top=ymax) for i in range(2)]

ymax = max([ax.get_ylim()[1] for ax in [ax1, ax2]])
[ax.set_ylim(top=ymax) for ax in [ax1, ax2]]
title = fr"{spot}. $ \Delta t $ = {freq_m}m. $ ( \bar{{\lambda}} , \delta ) = ({lambdas.mean():.2f} , {delta}) $. "
title += fr"$X={total_volume}$"
fig.suptitle(title)
fig.tight_layout()

fig.savefig(PLOT_FOLDER/'optimal_execution'/'oe_intraday_lambda.png')

In [ ]:
spot = 'ES'
train_len = 2
test_start_date = '2022-01-01'
freq_m = 15
base_vwap = get_vwap(spot=spot, train_len=train_months, test_start_date=test_start_date,
                     out_freq_m=freq_m)

fig, ax = plt.subplots(1)
ax.bar(range(len(base_vwap)), base_vwap)
ax.set_ylabel('Volume')
ax.set_xlabel('Time bin')
ax.set_title(f'VWAP Strategy for {spot}')
fig.tight_layout()
fig.savefig(PLOT_FOLDER/'optimal_execution'/'oe_vwap.png')

In [ ]:
results_list = []

train_months = 6
test_months = 3
smoothing = 'sma20'
half_life_s = 60*60

backtest_settings = BacktestSettings(
        train_len=train_months,
        test_len=test_months,
        bin_freq='10s',
        forecast_horizon_s=10,
        smoothing_str=smoothing,
        decay=half_life_s)

freq_m = 15
test_start_date = '2022-01-01'
do_bounds = True
spread_bps = 0

method = 'SLSQP'
tol = 1e-4

for spot in ['TU', 'GC', 'NQ']:
    spot = Spot(spot)
    future = FutureContract.from_spot(spot)

    stdev = get_std(spot, train_months, test_start_date)
    delta, lambdas = example_delta_lambdas(future, backtest_settings, freq_m=freq_m, date=test_start_date)

    trading_hours = len(lambdas)//(60/freq_m)
    num_bins = len(lambdas)

    base_vwap = get_vwap(spot=spot, train_len=train_months, test_start_date=test_start_date,
                         out_freq_m=freq_m)

    for total_volume in [0.025, 0.05, 0.1, 0.25] :

        vwap = base_vwap*total_volume

        open_strat = np.zeros(shape=num_bins)
        open_strat[0] = total_volume
        twap = np.ones(shape=num_bins)*total_volume/num_bins

        const_pim = PropagatorImpactModel(half_life_s=half_life_s, delta=delta,
                                          trading_hours=trading_hours, freq_m=freq_m, stdev=stdev,
                                          lambdas=lambdas.mean())

        twap_cost_placeholder = const_pim.func_to_optimise(twap)
        divisor = twap_cost_placeholder

        const_res = const_pim.get_optimal_volume(spread_bps=spread_bps, total_volume=total_volume,
                                                 do_bounds=do_bounds, tol=tol, divisor=divisor,
                                                 method=method)

        var_pim = PropagatorImpactModel(half_life_s=half_life_s, delta=delta,
                                        trading_hours=trading_hours, freq_m=freq_m, stdev=stdev,
                                        lambdas=lambdas)

        var_res = var_pim.get_optimal_volume(spread_bps=spread_bps, total_volume=total_volume,
                                             do_bounds=do_bounds, tol=tol/10, divisor=divisor,
                                             method=method)

        var_cost = var_pim.func_to_optimise(var_res.x)
        const_cost = var_pim.func_to_optimise(const_res.x)
        twap_cost = var_pim.func_to_optimise(twap)
        vwap_cost = var_pim.func_to_optimise(vwap)
        open_cost = var_pim.func_to_optimise(open_strat)

        if open_cost < var_cost or const_cost < var_cost:
            
            print(var_res.x.sum()/const_res.x.sum())
            const_pim.plot_volume(const_res.x, spread_bps)
            var_pim.plot_volume(var_res.x, spread_bps)

        results_list.append({'spot': spot, 'total_volume': total_volume,
                            'delta': delta, 'lambda_bar': lambdas.mean(),
                             'TWAP_cost': twap_cost,
                             'VWAP_cost': vwap_cost,
                             'Market Open_cost': open_cost,
                             'Constant_cost': const_cost,
                             'Intraday_cost': var_cost
                             })

In [ ]:
df = pd.DataFrame(results_list)
benchmarks = ['TWAP', 'VWAP', 'Market Open', 'Constant']
value_cols = benchmarks + ['Intraday']
cost_cols = [f"{col}_cost" for col in value_cols]
saving_cols = [f"{col}_savings" for col in benchmarks]

df.loc[:, cost_cols] = df[cost_cols] * 100*100

for bmark in benchmarks:
    df[f'{bmark}_savings'] = -100*(df['Intraday_cost'] - df[f'{bmark}_cost'])/df[f'{bmark}_cost']

df1 = df.copy()
df.loc[:, 'lambda_bar'] = df['lambda_bar'].apply(lambda x: np.round(x, 2))
df.loc[:, cost_cols] = df[cost_cols] .apply(lambda x: np.round(x, 4))
df.loc[:, saving_cols] = df[saving_cols] .apply(lambda x: np.round(x, 2))

init_cols = ['Spot', 'X', '$\delta$', r'$\bar{\lambda}$']
init_cols = [('', x) for x in init_cols]

new_cols = init_cols + [tuple(x.split('_')[::-1]) for x in df.columns[4:]]
df.columns = pd.MultiIndex.from_tuples(new_cols)
df.loc[:, df.columns[1:]] = df[df.columns[1:]].applymap(lambda x: str(x).rstrip('0'))

cost_df = df[['', 'cost']]
cost_df.columns = [col[1] for col in cost_df.columns]

savings_df = df[['', 'savings']]

savings_df.columns = [col[1] for col in savings_df.columns]
savings_df.loc[:, benchmarks] = savings_df[benchmarks].applymap(lambda x: x+r'\%').applymap(lambda x: x.replace(".\\", ".0\\"))


tables_folder = PLOT_FOLDER/'..'/'tables'/'optimal_execution'

cost_df.to_latex(tables_folder/'oe_costs.tex', index=False, escape=False)
savings_df.to_latex(tables_folder/'oe_savings.tex', index=False, escape=False)

savings_df

In [ ]:
x = df1.loc[4:,].loc[:, ['TWAP_savings','Market Open_savings','VWAP_savings']].min(axis=1).mean()
print(f"Savings of including intraday lambda compared to best benchmark of delta>0.1 --- {x:.2f}%\n")


x = df1.loc[4:,]['Constant_savings'].mean()
print(f"Savings of including intraday lambda compared to constant, for values of delta>0.1 --- {x:.2f}%\n")
x = df1['Constant_savings'].mean()
print(f"Savings of including intraday lambda compared to constant --- {x:.2f}%\n")

x = df1['Constant_savings'].max()
print(f"Max Savings of including intraday lambda compared to constant --- {x:.2f}%\n")
x = df1['Constant_savings'].mean()
print(f"Mean Savings of including intraday lambda compared to constant --- {x:.2f}%\n")


x = df1[['TWAP_savings','Market Open_savings','VWAP_savings']].max().max()
print(f"max saving for any benchmark and delta --- {x:.2f}%\n")